# Part 2: Modeling & Evaluation

## Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
from keras                   import regularizers
from keras.models            import Sequential
from keras.layers            import Dense
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics         import roc_auc_score, balanced_accuracy_score
from sklearn.metrics         import f1_score, confusion_matrix, recall_score
from sklearn.linear_model    import LogisticRegression
from sklearn.preprocessing   import StandardScaler, LabelBinarizer
from IPython.core.display    import display, HTML
from IPython.display         import display_html
sns.set(style = "white", palette = "deep")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

## Table Of Contents

- [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)
    

- [Establishing The Baseline](#Establishing-The-Baseline)
    
    
- [Preprocessing](#Preprocessing)
    - [Feature Engineering](#Feature-Engineering)
        - [Data Manipulation](#Data-Manipulation)
        - [Interaction Columns](#Interaction-Columns)
    - [Subset Definition](#Subset-Definition)
    - [Defining X & y Variables](#Defining-X-&-y-Variables)
    - [Train-Test Split](#Train-Test-Split)
    - [Scaling The Data](#Scaling-The-Data)

  
- [Modeling](#Modeling)
    - [Evaluation Functions](#Evaluation-Functions)
    - [Neural Networks](#Neural-Networks)
        - [Original Features](#Original-Features)
        - [Squared Features](#Squared-Features)
        - [Interaction Features](#Interaction-Features)
        
        
- [Evaluation](#Evaluation)
    - [Functions](#Functions)
    - [Dataframes](#Dataframes)
    - [Best Model Selection](#Best-Model-Selection)
    - [Plots](#Plots)
        - [Bar Chart](#Bar-Chart)
        - [ROC Curve](#ROC-Curve)

## Reading In The Data

In [ ]:
pulsar = pd.read_csv("../Data/pulsar_cleaned.csv")

### Overview

In [ ]:
# Checking the head of the data

pulsar.head()

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {pulsar.shape}")

In [ ]:
# Summary of column data types

pulsar.dtypes.value_counts()

In [ ]:
# Checking for null values

pulsar.isnull().sum()

[Top](#Table-Of-Contents)

## Establishing The Baseline

We need to know what our baseline accuracy is because that will give us an accuracy score to beat: if our accuracy is less than the baseline it means that our model is worse than guessing the category of a star.

In [ ]:
# Calculating the percentages of each class

round(pulsar["target_class"].value_counts(normalize = True)*100, 2)

In [ ]:
# Plotting the distribution of the classes

tick_labels = ["Non-Pulsar", "Pulsar"]

# Setting the figure size
plt.figure(figsize = (10,5))

# Plotting the graph
sns.countplot(pulsar["target_class"])

# Setting graph parameters
plt.title("Star Types", size = 18)
plt.xlabel("Star Type", size = 16)
plt.ylabel("Number Of Stars", size = 16)

# Making sure the only ticks are 0 and 1
plt.xticks(np.arange(0,2,1),
           labels = tick_labels,
           size   = 14)
plt.yticks(size = 14);

The data is _extremely_ imbalanced: this will make it difficult to model because the negative class (non-pulsar) is so much less frequent than the positive class (pulsar).

## Preprocessing

### Feature Engineering

#### Data Manipulation

While visualizing the data, we noticed that there are two columns that appeared to have a close to normal distribution.  As a result, we decided to square the values to transform them.

In [ ]:
# Squaring the `mean_ip` column

pulsar["mean_ip_squared"] = pulsar["mean_ip"].apply(lambda x: x**2)

# Squaring the `sd_ip` column

pulsar["sd_ip_squared"]   = pulsar["sd_ip"].apply(lambda x: x**2)

In [ ]:
# Making sure new columns were added

pulsar.columns

#### Interaction Columns

Based off of the heat map in the previous notebook, we noticed that there are some columns with very high correlations.  We felt that creating interaction columns, we would be emphasizing the correlation while also reducing the number of features.


The columns in particular are:


| Column 1   | Column 2   | Correlation |
|:-----------|:-----------|:-----------:|
| mean_ip    | sd_ip      | 0.55        |
| ex_kurt_ip | skew_ip    | 0.95        |
| mean_dmsnr | sd_dmsnr   | 0.80        |
| ex_kurt_ip | skew_dmsnr | 0.92        |


We chose these columns in particular because they allows us to reduce the number of features that will end up going into the model.  However, we decided not to get ride of the eight original features: we will end up with sub-sets of the model we will model on.

In [ ]:
# Defining the `mean_ip` * `sd_ip` colum
# Defining the `ex_kurt_ip` * `skew_ip` colum

pulsar["mean_*_sd_ip"]     = pulsar["mean_ip"] * pulsar["sd_ip"]
pulsar["exkurt_*_skew_ip"] = pulsar["ex_kurt_ip"] * pulsar["skew_ip"]

# Defining the `mean_dmsnr` * `sd_dmsnr` colum
# Defining the `ex_kurt_dmsnr` * `skew_dmsnr` colum

pulsar["mean_*_sd_dmsnr"]     = pulsar["mean_dmsnr"] * pulsar["sd_dmsnr"]
pulsar["exkurt_*_skew_dmsnr"] = pulsar["ex_kurt_dmsnr"] * pulsar["skew_dmsnr"]

In [ ]:
# Checking to make sure columns were created

pulsar.columns

[Top](#Table-Of-Contents)

### Subset Definition

We decided that because of the extreme imbalance of the classes a feed forward neural network is the best approach to predicting the presence of a pulsar.

Because of the way we set up the interaction columns and manipulated the data, we decided that we are best off with three subsets: the original features, original features with squared columns, and the interaction columns.  We have to define the subsets before we define our X and y variables.

In [ ]:
# List of the original features

original_features = ["mean_ip", "sd_ip", "ex_kurt_ip", "skew_ip",
                     "mean_dmsnr", "sd_dmsnr", "ex_kurt_dmsnr", "skew_dmsnr",
                     "target_class"]

# List of the original features with `mean_ip` and `sd_ip` squared

manipulated_features = ["mean_ip_squared", "sd_ip_squared", "ex_kurt_ip", "skew_ip",
                        "mean_dmsnr", "sd_dmsnr", "ex_kurt_dmsnr", "skew_dmsnr",
                        "target_class"]

# List of the interaction features

interaction_features = ["mean_*_sd_ip", "exkurt_*_skew_ip", "mean_*_sd_dmsnr",
                        "exkurt_*_skew_dmsnr", "target_class"]

In [ ]:
# Defining a subset with the original features

pulsar_og = pulsar[original_features]

# Defining a subset with the original/squared features

pulsar_sq = pulsar[manipulated_features]

# Defining a subset with the interaction features

pulsar_if = pulsar[interaction_features]

### Defining X & y Variables

From this point on `X` and `y` will refer to the original features, `_sq` will refer to the dataframe with the squared features, and `_if` will refer to the dataframe with interaction features.

In [ ]:
# X and y for the original

X = pulsar_og.drop("target_class", axis = 1)
y = pulsar_og["target_class"]

# X and y for the original/squared features

X_sq = pulsar_sq.drop("target_class", axis = 1)
y_sq = pulsar_sq["target_class"]

# X and y for the interaction features

X_if = pulsar_if.drop("target_class", axis = 1)
y_if = pulsar_if["target_class"]

### Train-Test Split

The purpose of the train-test split is to split up our data so that we can reserve an unseen portion of it to test our model on.  Additionally, we will be setting a random state for reproducability and we will stratify on `y` so that the distribution of classes is preserved.

In [ ]:
# Instantiating the scaler

ss = StandardScaler()

In [ ]:
# Transforming here makes it easier than
# scaling after the train-test split

X_ss = ss.fit_transform(X)
X_sq_ss = ss.fit_transform(X_sq)
X_if_ss = ss.fit_transform(X_if)

In [ ]:
# For the original data

X_train, X_test, y_train, y_test = train_test_split(X_ss,
                                                    y,
                                                    random_state = 42,
                                                    stratify     = y)

# For the original data with squared features

X_sq_train, X_sq_test, y_sq_train, y_sq_test = train_test_split(X_sq_ss,
                                                                y_sq,
                                                                random_state = 42,
                                                                stratify     = y_sq )

# For the dataframe with interaction features

X_if_train, X_if_test, y_if_train, y_if_test = train_test_split(X_if_ss,
                                                                y_if,
                                                                random_state = 42,
                                                                stratify     = y_if)

[Top](#Table-Of-Contents)

## Modeling

### Evaluation Functions

A confusion matrix gives us a summary of how our model classified the test data: it compares the true and predicted y values so that we can see how the model performed on each class.


Each confusion matrix is set up the same way:

|                     | Predicted Positive | Predicted Negative |
|:--------------------|:------------------:|:------------------:|
| **Actual Positive** | True Negative      | False Negative     |
| **Actual Negative** | False Positive     | True Positive      |

In [ ]:
# We converted the confusion matrix to a dataframe to make it easier to read

def create_confusion_matrix(y, y_preds):
    cm     = confusion_matrix(y, y_preds)
    matrix = pd.DataFrame(cm, 
                          columns = ["Predicted Non-Pulsar", "Predicted Pulsar"], 
                          index   = ["Actual Non-Pulsar", "Predicted Pulsar"])
    return matrix

We will generate two scores based off of the confusion matrix: specificity and sensitivity; we will go into what these scores measure in a few cells.

In [ ]:
# Calculating specificity from a confusion matrix

def specificity(y, y_pred):
    cm          = confusion_matrix(y, y_pred)  
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    return specificity

In addition to metric scores, we will also calculate an ROC-AUC score.  The ROC (receiver operating characteristic) shows us a binary classification model's ability to distinguish between two classes; we will plot this curve for the best model as determined by metric scores.

These images from [GreyAtom](https://medium.com/greyatom/lets-learn-about-auc-roc-curve-4a94b4d88152) illustrates the AUC-ROC well:

<img src = "../Images/ROC_AUC 0.8 0.9.png" alt = "high auc_roc scores" height = "350" width = "350">

<img src = "../Images/ROC_AUC 0.5 0.7.png" alt = "low auc_roc scores"  height = "350" width = "350">


Accuracy is not the most informative score for us: it just counts how many posts were classified correctly.  Instead, we want to look at performance regarding the predicted positives (pulsars) and negatives (non-pulsars).  We will use the following scores:

| Metric                | Definition                                                       | Scale    |
|:----------------------|:-----------------------------------------------------------------|:---------|
| **Balanced Accuracy** | The average of the recall on each class                          | 0 to 1   | 
| **Specificity**       | How many negative predictions are correct                        | 0 to 1   | 
| **Sensitivity**       | How many positive predictions are correct (also known as recall) | 0 to 1   | 
| **F1 Score**          | Accuracy that takes into account the specificity & sensitivity   | 0 to 1   | 
| **ROC-AUC Score**     | A measure of the model's ability to distinguish classes          | 0.5 to 1 |

One of the parameters for the F1 and ROC-AUC scores is called `average` which, as the name suggests, controls how the averages are calculate.  The default is `binary`, but because our classes are extremely unbalanced we will use the value `macro` which calculates the score on each class but does _not_ apply any kind of weighting; doing this ignores the class imbalance.

In [ ]:
# Generating the 6 metric evaluation

def generate_model_eval(y, y_pred):
    print(f"The balanced accuracy score is: {round(balanced_accuracy_score(y, y_pred), 5)}")
    print(f"The specificity score is      : {round(specificity(y, y_pred), 5)}")
    print(f"The sensitivity score is      : {round(recall_score(y, y_pred), 5)}")
    print(f"The F1 score is               : {round(f1_score(y, y_pred, average = 'macro'), 5)}")
    print(f"The ROC-AUC score is          : {round(roc_auc_score(y, y_pred, average = 'macro'), 5)}")

[Top](#Table-Of-Contents)

### Neural Networks

<img src = "../Images/neural_network.jpg" alt = "simple neural network" height = "350" width = "400">


A neural network is a modeling technique that consists of a number of simple but highly interconnected elements or nodes or neurons 
which are organized in layers.


- **Input layer**: the number of features in the data; in our case, the number of columns


- **Hidden layer(s)**: the "middle" features and are called hidden because they are neither the input nor output.  The hidden layer(s) are where the actual computations take place: using weights and biases they modify linear models.  The hidden layers also have activation functions which modify the output in some way; the most common activation function is ReLU (rectified linear unit) which forces the hidden layer to be positive.


- **Output layer**: what the actual result of the computation is and they determine what the results look like.  In the case of regression, it is a number whereas in classification the output is a probability.


If we are not careful, our neural networks can become _extremely_ complicated and overfit to the training data.  To combat that, we will regularize the models.


The models will have the following structure:

- A dense network

- An input layer

- Hidden layers
    - ReLU activation

- Single output layer
    - Sigmoid activation

#### Original Features

In [ ]:
# Defining the input size

input_shape = X_train.shape[1]

# Instantiating the model

of_model = Sequential()

# Adding a primary dense layer
# L1 regularization is not really used with NNs

of_model.add(Dense(6,
                   input_dim          = input_shape,
                   activation         = "relu",
                   kernel_regularizer = regularizers.l2(0.001)))

# Adding an output layer
# L1 regularization is not really used with NNs

of_model.add(Dense(1,
                   activation         = "sigmoid",
                   kernel_regularizer = regularizers.l2(0.001)))

# Compiling the model

of_model.compile(loss      = "binary_crossentropy",
                 optimizer = "adam",
                 metrics   = ["acc"])

In [ ]:
# Fitting the model

of_model.fit(X_train,
             y_train,
             epochs     = 150,
             batch_size = 100,
             verbose    = 0)

In [ ]:
# Generating training predictions

of_train_preds = of_model.predict_classes(X_train,
                                          batch_size = 100,
                                          verbose    = 0).ravel()

# Generating test predictions

of_test_preds = of_model.predict_classes(X_test,
                                         batch_size = 100,
                                         verbose    = 0).ravel()

In [ ]:
# Evaluating the training predictions

generate_model_eval(y_train, of_train_preds)

In [ ]:
# Evaluating the test predictions

generate_model_eval(y_test, of_test_preds)

In [ ]:
create_confusion_matrix(y_test, of_test_preds)

#### Squared Features

In [ ]:
# Defining the input size

input_shape = X_sq_train.shape[1]

# Instantiating the model

sq_model = Sequential()

# Adding a primary dense layer
# L1 regularization is not really used with NNs

sq_model.add(Dense(6,
                   input_dim          = input_shape,
                   activation         = "relu",
                   kernel_regularizer = regularizers.l2(0.001)))

# Adding an output layer
# L1 regularization is not really used with NNs

sq_model.add(Dense(1,
                   activation         = "sigmoid",
                   kernel_regularizer = regularizers.l2(0.001)))

# Compiling the model

sq_model.compile(loss      = "binary_crossentropy",
                 optimizer = "adam",
                 metrics   = ["acc"])

In [ ]:
# Fitting the model

sq_model.fit(X_sq_train,
             y_sq_train,
             epochs     = 150,
             batch_size = 100,
             verbose    = 0)

In [ ]:
# Generating training predictions

sq_train_preds = sq_model.predict_classes(X_sq_train,
                                          batch_size = 100,
                                          verbose    = 0).ravel()

# Generating test predictions

sq_test_preds = sq_model.predict_classes(X_sq_test,
                                         batch_size = 100,
                                         verbose    = 0).ravel()

In [ ]:
# Evaluating the training predictions

generate_model_eval(y_sq_train, sq_train_preds)

In [ ]:
# Evaluating the test predictions

generate_model_eval(y_sq_test, sq_test_preds)

In [ ]:
create_confusion_matrix(y_sq_test, sq_test_preds)

#### Interaction Features

In [ ]:
# Defining the input size

input_shape = X_if_train.shape[1]

# Instantiating the model

if_model = Sequential()

# Adding a primary dense layer
# L1 regularization is not really used with NNs

if_model.add(Dense(6,
                   input_dim          = input_shape,
                   activation         = "relu",
                   kernel_regularizer = regularizers.l2(0.001)))

# Adding an output layer
# L1 regularization is not really used with NNs

if_model.add(Dense(1,
                   activation         = "sigmoid",
                   kernel_regularizer = regularizers.l2(0.001)))

# Compiling the model

if_model.compile(loss      = "binary_crossentropy",
                 optimizer = "adam",
                 metrics   = ["acc"])

In [ ]:
# Fitting the model

if_model.fit(X_if_train,
             y_if_train,
             epochs     = 150,
             batch_size = 100,
             verbose    = 0)

In [ ]:
# Generating training predictions

if_train_preds = if_model.predict_classes(X_if_train,
                                          batch_size = 100,
                                          verbose    = 0).ravel()

# Generating test predictions

if_test_preds = if_model.predict_classes(X_if_test,
                                         batch_size = 100,
                                         verbose    = 0).ravel()

In [ ]:
# Evaluating the training predictions

generate_model_eval(y_if_train, if_train_preds)

In [ ]:
# Evaluating the test predictions

generate_model_eval(y_if_test, if_test_preds)

In [ ]:
create_confusion_matrix(y_if_test, if_test_preds)

## Evaluation

### Functions

In [ ]:
# Plotting a bar plot of a model's scores

def plot_scores(df, column, label):
    
    # Setting the figure size
    plt.figure(figsize   = (15,5),
               facecolor = "white")
    
    # Plotting the bar plot
    sns.barplot(x    = df.index,
                y    = column,
                data = df)
    
    # Setting the baseline line
    plt.axhline(9.16, 
                color = "black")
    
    # Setting graph parameters
    plt.title(label, size = 20)
    plt.xlabel("Model", size = 18)
    plt.ylabel("Score", size = 18)
    plt.xticks(size  = 14)
    plt.yticks(ticks = np.arange(0,110,10), 
               size  = 14)

In [ ]:
# Generating scores and plotting the ROC curve

def roc(model_prob, X_test, y_test, y_pred, title):
    
    # Calculating probabilities
    model_prob    = [i[0] for i in model_prob.predict_proba(X_test)]
    
    # Creating a dataframeout of the true values & probas
    model_pred_df = pd.DataFrame({"true_values": y_test,
                                  "pred_probs" : model_prob})

    # Setting threshold values    
    thresholds = np.linspace(0, 1, 500) 
    
    # Calculating the sensitivity
    def true_positive_rate(df, true_col, pred_prob_col, threshold):
        true_positive  = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
        false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
        return true_positive / (true_positive + false_negative)
    
    # Calculating the false positives
    def false_positive_rate(df, true_col, pred_prob_col, threshold):
        true_negative  = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
        false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
        return 1 - (true_negative / (true_negative + false_positive))
    
    # Calculating the sensitivity and false positives for each point in the threhold
    tpr_values = [true_positive_rate(model_pred_df, 
                                     "true_values", 
                                     "pred_probs", 
                                     prob) for prob in thresholds]
    fpr_values = [false_positive_rate(model_pred_df, 
                                      "true_values", 
                                      "pred_probs", 
                                      prob) for prob in thresholds]

    # Setting up the graph
    plt.figure(figsize   = (13,7),
               facecolor = "white")
    
    # Plotting the predicted
    plt.plot(fpr_values, 
             tpr_values,
             color = "darkorange",
             label = "ROC Curve")
    
    # Setting the baseline
    plt.plot(np.linspace(0, 1, 500),
             np.linspace(0, 1, 500),
             color     = "darkblue",
             label     = "Baseline"),
    
    # Setting model parameters
    plt.title(title, fontsize = 18)
    plt.ylabel("Sensitivity", size = 16)
    plt.xlabel("1 - Specificity", size = 16)
    plt.xticks(size = 14)
    plt.yticks(size = 14)
    plt.legend(bbox_to_anchor = (1.04, 1), 
               loc            = "upper left",
               fontsize       = 16)
    plt.tight_layout()
    
# The code was modified from code written by Matt Brems during our lesson on classification metrics.

In [ ]:
# This function allows for dataframes to be displayed side-by-side

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table', 'table style="display:inline"'), raw = True)

### Dataframes

In [ ]:
# Metrics scores

balanced_accuracy = [balanced_accuracy_score(y_test, of_test_preds),
                     balanced_accuracy_score(y_sq_test, sq_test_preds),
                     balanced_accuracy_score(y_if_test, if_test_preds)]

specificity       = [specificity(y_test, of_test_preds),
                     specificity(y_sq_test, sq_test_preds),
                     specificity(y_if_test, if_test_preds)]

sensitivity       = [recall_score(y_test, of_test_preds),
                     recall_score(y_sq_test, sq_test_preds),
                     recall_score(y_if_test, if_test_preds)]

f1_score          = [f1_score(y_test, of_test_preds),
                     f1_score(y_sq_test, sq_test_preds),
                     f1_score(y_if_test, if_test_preds)]

rocauc_score      = [roc_auc_score(y_test, of_test_preds),
                     roc_auc_score(y_sq_test, sq_test_preds),
                     roc_auc_score(y_if_test, if_test_preds)]

In [ ]:
# Converting the lists into dataframes

# A dataframe for the scores

scores = pd.DataFrame(data    = [balanced_accuracy, specificity,
                                 sensitivity, f1_score],
                      columns = ["Original Features", "Squared Features", 
                                 "Interaction Features"],
                      index   = ["Balanced Accuracy", "Specificity", 
                                 "Sensitivity", "F1 Score"])

In [ ]:
# Making sure the dataframe looks right

scores

### Best Model Selection

### Plots

#### Bar Chart

In [ ]:
plot_scores(df     = scores*100,
            column = "Squared Features",
            label  = "Squared Features")

#### ROC Curve

In [ ]:
# Generating the ROC-AUC score

rocauc_score = round(roc_auc_score(y_sq_test, sq_test_preds), 5)

# Plotting the ROC curve with the score in the title

roc(model_prob = sq_model,
    X_test     = X_sq_test,
    y_test     = y_sq_test,
    y_pred     = sq_test_preds,
    title      = f"ROC For Neural Network With A Score Of {rocauc_score}")